In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams["axes.unicode_minus"] = False
import seaborn as sns
import gc
from tqdm.auto import tqdm
tqdm.pandas()

import time
import datetime
import chinese_calendar as calendar  # !pip install chinesecalendar
from chinese_calendar import is_workday, is_holiday
import warnings
warnings.filterwarnings('ignore')

import uuid
pd.set_option("display.max_columns", None)

import random
import os

from sqlalchemy import create_engine
import psycopg2
import sqlalchemy

def to_float16(df):
    float64_columns = df.columns[df.dtypes == 'float64']
    df[float64_columns] = df[float64_columns].astype(np.float16)
    return df

def to_int16(df):
    int16_columns = df.columns[df.dtypes == 'int64']
    df[int16_columns] = df[int16_columns].astype(np.float16)
    return df
import datetime    


In [2]:
gbk_data_path = '/home/AI/DATA'

year_list = [2018, 2019, 2020]
gbk_year_data_name = 'GBK_{}_DATA'


user = "analy"
pwd = "An@postgres.2020"
host = "10.3.78.136"
# host = "10.3.78.134"
port = "9432"

dbname = "zwd_predict"
shema = "ods"

# engine = create_engine('mysql+mysqlconnector://[user]:[pass]@[host]:[port]/[dbname]', echo=False)
# 以上为引擎创建格式

con = 'postgresql+psycopg2://'+user+':'+pwd+'@'+host+':'+port+'/'+dbname

In [ ]:
#  show_database_sql = "select datname from pg_database"
# show_tables_sql = "select tablename from pg_tables where schemaname='ods'order by tablename"


# sql = 'select statis_id, ljpym, stn_name, daoda_time, chufa_time, arr_name, arrival0, depart0, arr_delay, dpt_delay, track, stn_flag,arr_er,dpt_er from ods."gbk_kypx_detail_2020_jggt"'
# sql = "select * from ods.gbk_kypx_detail_2020"
# sql = "select statis_id, ljpym, stn_name, daoda_time, chufa_time, arr_name, arrival0, depart0, arr_delay, dpt_delay, track, stn_flag  from ods.gbk_kypx_detail_2020 limit 100"
# sql = "select statis_id, ljpym, stn_name, daoda_time, chufa_time, arr_name, arrival0, depart0, arr_delay, dpt_delay, track, stn_flag  from ods.gbk_kypx_detail_2019 where date_part('month' ,cast (daoda_time as timestamp)) in("+part_month+")"
# sql = "select  statis_id, ljpym, stn_name, daoda_time, chufa_time, arr_name, arrival0, depart0, arr_delay, dpt_delay, track, stn_flag,xm,arr_er,dpt_er from ods.gbk_kypx_detail_2020_jh  where substring(arr_name, 1 ,1 ) in ('C','D','G') and XM='沪蓉线' order by statis_id,arrival0;"
sql = "select  statis_id, ljpym, stn_name, daoda_time, chufa_time, arr_name, arrival0, depart0, arr_delay, dpt_delay, track, stn_flag,xm,arr_er,dpt_er from ods.gbk_kypx_detail_2020_jh  where substring(arr_name, 1 ,1 ) in ('C','D','G') and statis_id in (select  distinct   statis_id from ods.gbk_kypx_detail_2020_jh  where substring(arr_name, 1 ,1 ) in ('C','D','G') and XM='沪昆高铁');"
df = pd.read_sql(sql, con)


df.columns = [i.upper() for i in df.columns]

print(df.info(memory_usage='deep'))
df


In [ ]:
# # (df['ARR_NAME'].str[-1])%2==0
# q=df['ARR_NAME'].str[-1]
# q=q.astype(int)
# df['sx']=q

In [ ]:
# # df=df[df['sx']%2==0]
# # df
# df=df[df['sx']%2==1]
# df

In [ ]:
s=df[df['STN_NAME'].str[-3:]=='线路所']
s=s.index

In [ ]:
df=df[~df.index.isin(s)]
df

In [ ]:
s=df.drop_duplicates(['STATIS_ID'],keep='first')
len(s)

In [ ]:

df['jhtz']=np.where(df['ARRIVAL0']>df['DEPART0'],2,np.where(round(
    (pd.to_datetime(df['DEPART0']) -
     pd.to_datetime(df['ARRIVAL0'])).dt.seconds)>50,1,0))
df['sjtz']=np.where(df['DAODA_TIME']>df['CHUFA_TIME'],2,np.where(round(
    (pd.to_datetime(df['CHUFA_TIME']) -
     pd.to_datetime(df['DAODA_TIME'])).dt.seconds)>50,1,0))

In [ ]:
# error=df[(df['jhtz']!=df['sjtz']) & ( (df['STN_FLAG']!='1') & (df['STN_FLAG']!='2') )]
error=df[ ((df['jhtz']==2) | (df['sjtz']==2)) & ( (df['STN_FLAG']!='1') & (df['STN_FLAG']!='2') ) ]
error.drop_duplicates(['STATIS_ID'],keep='first',inplace=True)
error

In [ ]:
df=df[~df['STATIS_ID'].isin(error['STATIS_ID'])]
df

In [ ]:
df=df[(pd.to_datetime(df['DAODA_TIME'])-pd.to_datetime(df['CHUFA_TIME']))<'00:01:40']
df

In [ ]:
# id_qc=df[((df['ARR_ER']<=5)|(df['DPT_ER']<=5))& ((df['ARR_ER']-df['DPT_ER'])<=20)]
id_qc=df[np.abs(df['ARR_ER']<5)|np.abs(df['DPT_ER']<5)]
id_qc=id_qc[np.abs(id_qc['ARR_ER']-id_qc['DPT_ER'])>20]

In [ ]:
id_qc.drop_duplicates(['STATIS_ID'],keep='first',inplace=True)
len(id_qc)

In [ ]:
df=df[~df['STATIS_ID'].isin(id_qc['STATIS_ID'])]

In [ ]:
del df['ARR_ER']
del df['DPT_ER']

In [ ]:
df

In [ ]:
df = df.groupby('STATIS_ID').filter(lambda x: len(x['ARR_NAME'].values[0]) < 6) 
df = df[df['ARR_NAME'].map(lambda x: str(x[0]) in 'GD')]

In [ ]:
df.drop_duplicates(subset=df.columns[1:],keep='first',inplace=True)
df

In [ ]:
bad_id = df[df['STN_NAME'].isnull()]['STATIS_ID'].tolist()
df = df[~df['STATIS_ID'].isin(bad_id)]
del bad_id

df

In [ ]:
df.isnull().sum()

In [ ]:
df[df['STN_FLAG'] == 0]['ARR_DELAY1'] = 0
df[df['STN_FLAG'] == 1]['DPT_DELAY1'] = 0

df

In [ ]:
# 基于行程ID和到达时间排序
df['ARR_DELAY'] = df['ARR_DELAY'] * 60
df['DPT_DELAY'] = df['DPT_DELAY'] * 60
df.sort_values(['STATIS_ID','DAODA_TIME'], inplace=True, ascending=[True,True])
df.reset_index(drop=True, inplace=True)

# 构建"上一站出发晚点时间"字段
df['LAST_DPT_DELAY']=0
for i in range(len(df)):
    if i==0:
        df['LAST_DPT_DELAY'].iat[i]=0
    else:
        #print(i)
        if df['STATIS_ID'].iat[i]==df['STATIS_ID'].iat[i-1]:
            df['LAST_DPT_DELAY'].iat[i]=df['DPT_DELAY'].iat[i-1]
        else:
            df['LAST_DPT_DELAY'].iat[i]=0
#df=df.head(10000)
arr_delay_types = ['到达正点','到达初始晚点','到达连带晚点']           
dpt_delay_types = ['出发正点','出发初始晚点','出发连带晚点']
df['ARR_DELAY_TYPE']='0'
df['DPT_DELAY_TYPE']='0'
for i in range(len(df)):
    if i==0:
        if df['ARR_DELAY'].iat[i]>240:
            df['ARR_DELAY_TYPE'].iat[i]=arr_delay_types[1]
            df['DPT_DELAY_TYPE'].iat[i]=dpt_delay_types[2]
        else:
            df['ARR_DELAY_TYPE'].iat[i]=arr_delay_types[0]
            df['DPT_DELAY_TYPE'].iat[i]=dpt_delay_types[0]
    else:
       # print(i)
        if df['STATIS_ID'].iat[i]==df['STATIS_ID'].iat[i-1]:
            if df['DPT_DELAY_TYPE'].iat[i-1]==dpt_delay_types[0]:
                if df['ARR_DELAY'].iat[i]>240:
                    df['ARR_DELAY_TYPE'].iat[i]=arr_delay_types[1]
                else:
                    df['ARR_DELAY_TYPE'].iat[i]=arr_delay_types[0]
            else:
                if df['ARR_DELAY'].iat[i]>0:
                    df['ARR_DELAY_TYPE'].iat[i]=arr_delay_types[2]
                else:
                    df['ARR_DELAY_TYPE'].iat[i]=arr_delay_types[0]          
            if df['ARR_DELAY_TYPE'].iat[i]==arr_delay_types[0]:
                if df['DPT_DELAY'].iat[i]>240:
                    df['DPT_DELAY_TYPE'].iat[i]=dpt_delay_types[1]
                else:
                    df['DPT_DELAY_TYPE'].iat[i]=dpt_delay_types[0]
            else:
                if df['DPT_DELAY'].iat[i]>0:
                    df['DPT_DELAY_TYPE'].iat[i]=dpt_delay_types[2]
                else:
                    df['DPT_DELAY_TYPE'].iat[i]=dpt_delay_types[0]
        else:
            if df['ARR_DELAY'].iat[i]>240:
                df['ARR_DELAY_TYPE'].iat[i]=arr_delay_types[1]
                df['DPT_DELAY_TYPE'].iat[i]=dpt_delay_types[2]
            else:
                df['ARR_DELAY_TYPE'].iat[i]=arr_delay_types[0]
                df['DPT_DELAY_TYPE'].iat[i]=dpt_delay_types[0]
        
            
    
                    
            
        
            
            
            
            
# df['LAST_DPT_DELAY'] = df['DPT_DELAY'].shift(1)
# df['LAST_DPT_DELAY'] = np.where(df['STATIS_ID'].shift(1) == df['STATIS_ID'], df['LAST_DPT_DELAY'], 0)  # 始发站上一站出发晚点时间置0

# 构建"到达晚点类型"字段
#conditions = [df['ARR_DELAY'] == 0, df['ARR_DELAY'] < 0, df['ARR_DELAY'] > 0]
#choices = ['到达正点','早到','到达初始晚点']  # 先将晚点全部设为初始晚点，后续再部分更新为连带晚点
#df['ARR_DELAY_TYPE'] = np.select(conditions, choices, default='unkown')

# 构建"出发晚点类型"字段
#conditions = [df['DPT_DELAY'] == 0, df['DPT_DELAY'] < 0, df['DPT_DELAY'] > 0]
#choices = ['出发正点','早发','出发初始晚点']
#df['DPT_DELAY_TYPE'] = np.select(conditions, choices, default='unkown')

In [ ]:

# # 标记"到达连带晚点"
# index = np.where((df['ARR_DELAY'] > 0) & (df['LAST_DPT_DELAY'] > 0))[0]
# df['ARR_DELAY_TYPE'][index] = '到达连带晚点'
# df['ARR_DELAY_TYPE'].value_counts()


In [ ]:
# # 标记"出发连带晚点"
# index = np.where((df['DPT_DELAY'] > 0) & (df['ARR_DELAY'] > 0))[0]
# df['DPT_DELAY_TYPE'][index] = '出发连带晚点'
# print(df['DPT_DELAY_TYPE'].value_counts())

# del df['LAST_DPT_DELAY']  # 删除"上一站出发晚点时间"字段
# df


In [ ]:
df.sort_values(['STATIS_ID', 'DAODA_TIME'], inplace=True, ascending=[True, True])

df['LJPYM'] = df['LJPYM'].fillna(method='ffill', axis=0)
new_col = ['STATIS_ID', 'TYPE']

df_type = df.groupby('STATIS_ID')['LJPYM'].nunique().reset_index()
df_type.columns = new_col

df = pd.merge(df, df_type, on = 'STATIS_ID')
df.loc[df['TYPE'] > 1, 'TYPE'] = 'ZT'
df.loc[df['TYPE'] == 1, 'TYPE'] = 'GN'

del df['LJPYM']
df

In [ ]:
# 当前站到下一站的区间实际运行时间
df['SJ_YX_TIME']=0
df['TD_YX_TIME']=0
for i in range(len(df)):
    if i==0:
        df['SJ_YX_TIME'].iat[i]=0
        df['TD_YX_TIME'].iat[i]=0
    else:
        print(i)
        if df['STATIS_ID'].iat[i]==df['STATIS_ID'].iat[i-1]:
#             print(df['DAODA_TIME'].iat[i+1])
# #             print(pd.to_datetime(df['DAODA_TIME'].iat[i+1]).time)
#             print(pd.to_datetime(df['DAODA_TIME'].iat[i+1])-pd.to_datetime(df['CHUFA_TIME'].iat[i]))
            df['SJ_YX_TIME'].iat[i]=(pd.to_datetime(df['DAODA_TIME'].iat[i])-pd.to_datetime(df['CHUFA_TIME'].iat[i-1])).seconds
            df['TD_YX_TIME'].iat[i]=(pd.to_datetime(df['ARRIVAL0'].iat[i])-pd.to_datetime(df['DEPART0'].iat[i-1])).seconds
        else:
            df['SJ_YX_TIME'].iat[i]=0
            df['TD_YX_TIME'].iat[i]=0
# df['SJ_YX_TIME'] = (pd.to_datetime(df['DAODA_TIME'].shift(-1)) -
#                     pd.to_datetime(df['CHUFA_TIME'])).dt.seconds
# # 当前站到下一站的区间图定运行时间
# df['TD_YX_TIME'] = (pd.to_datetime(df['ARRIVAL0'].shift(-1)) -
#                     pd.to_datetime(df['DEPART0'])).dt.seconds
# # 标志位，后面用于过滤每条行程的最后一站数据
# df['IS_SAME'] = (df['STATIS_ID'].shift(1) == df['STATIS_ID']).shift(-1)

In [ ]:
df['SJ_YX_TIME/TD_YX_TIME']=df['SJ_YX_TIME'] / df['TD_YX_TIME']

In [ ]:
# ## 对【实际运行时间/计划运行时间】的比值进行四分位数统计

# # 过滤每条行程的最后一站数据
# # df1 = df[df['IS_SAME'] == True]

# df1 = df[['STATIS_ID', 'SJ_YX_TIME', 'TD_YX_TIME','STN_NAME']]
# # 增加比值列
# df1['SJ_YX_TIME/TD_YX_TIME'] = df1['SJ_YX_TIME'] / df1['TD_YX_TIME']
# # 筛选发生了区间吸收的数据
# df1 = df1[df1['SJ_YX_TIME/TD_YX_TIME'] >1]
# # 计算区间IQR
# k = 3
# Q1 = df1['SJ_YX_TIME/TD_YX_TIME'].quantile(0.25)
# Q3 = df1['SJ_YX_TIME/TD_YX_TIME'].quantile(0.75)
# QJ_IQR = round(Q3 + k * (Q3 - Q1), 2)
# print('Q1',Q1, 'Q3',Q3, 'QJ_IQR',QJ_IQR)
# # 筛选出比值小于IQR的id集合
# remove_ids1 = set(df1[df1['SJ_YX_TIME/TD_YX_TIME'] > QJ_IQR]['STATIS_ID'].values)
remove_ids1=df[(df['TD_YX_TIME']>=180) &  (df['SJ_YX_TIME/TD_YX_TIME']>=3)]

In [ ]:
df1.drop_duplicates('STATIS_ID',keep='first',inplace=True)

In [ ]:
# df2 = df[df['STATIS_ID'].isin(remove_ids1)]
# df2

In [ ]:
# df2[(df2['SJ_YX_TIME/TD_YX_TIME']>1.59)]

In [ ]:
len(remove_ids1)

In [ ]:
id=df.drop_duplicates('STATIS_ID',keep='first')

In [ ]:
df = df[~df['STATIS_ID'].isin(remove_ids1)]
df

In [ ]:
# # 过滤区间吸收异常或车站吸收异常的行程
# remove_ids = remove_ids1
# df = df[~df['STATIS_ID'].isin(remove_ids)]

# # 验证，交集应该为空
# assert set(df['STATIS_ID'].values) & set(remove_ids) == set()

# # 删除无用列
# # df.drop(df[['SJ_YX_TIME', 'TD_YX_TIME', 'IS_SAME']], axis=1, inplace=True)
# # del df1,df2
# df.shape

In [ ]:
# arr_up_threshold = arr_threshold2
# dpt_up_threshold = dpt_threshold2
# print('arr_up_threshold:',arr_up_threshold)
# print('dpt_up_threshold:',dpt_up_threshold)

In [ ]:

# id_all = set(df['STATIS_ID'])

# set1 = set(df['STATIS_ID'].values[np.where(np.abs(df['ARR_DELAY1'].values) > arr_up_threshold)[0]])
# set2 = set(df['STATIS_ID'].values[np.where(np.abs(df['DPT_DELAY1'].values) > dpt_up_threshold)[0]])
# id_remove = set1.union(set2)
# id_keep = id_all.difference(id_remove)

# df = df[df['STATIS_ID'].isin(id_keep)]
# assert len(np.where(np.abs(df['ARR_DELAY1'].values) > arr_up_threshold)[0]) + len(np.where(np.abs(df['DPT_DELAY1'].values) > dpt_up_threshold)[0]) == 0

# # release
# del df_tmp

# print(df.columns)
# print(df.shape)

In [ ]:
# df_ajb = pd.read_csv(os.path.join(gbk_data_path, '2020-AJB_DATA.csv'))

# df_ajb.dropna(how='any', inplace=True)

# print(df_ajb.info())
# df_ajb

In [ ]:
# ids_list = df_ajb['STATIS_ID'].unique().tolist()
# print(len(ids_list))

# df = df[~df['STATIS_ID'].isin(ids_list)]
# df

In [ ]:
# df['FORMER_STN']
# df['FORMER_STN'] = df['STN_NAME'].shift(1)  # 上一站

In [ ]:
# sql = "select * from ods.zwd_xd_xb"
# df_xlqj = pd.read_sql(sql, con)
# print(df_xlqj.shape)

# df_xlqj = {(k1,k2): v for k1, k2, v in df_xlqj[['q_czm','h_czm','xm']].values}

# def assign_xm(x):
#     k1 = x['FORMER_STN']
#     k2 = x['STN_NAME']
  
#     try:
#         return df_xlqj[(k1, k2)]
#     except:
#         try: 
#             return df_xlqj[(k2, k1)]
#         except:
#             return '0' 

        
# df['FORMER_STN'] = df['STN_NAME'].shift(1)  # 上一站
# df.loc[:, "xm"] = df.progress_apply(assign_xm, axis=1)

# df

In [ ]:
# df1=df
# df1

In [ ]:
# # xm_var = df['xm'].unique().tolist()
# # xm_var
# # print([i for i in xm_var if '京沪' in i ])

# # xm = ['京广高速','京广高速京西联']
# # df1 = df[df['xm'].map(lambda x: str(x[0:3]) in '京广高速')]
# df = df[df['xm'].str[0:4]=='京广高速']
# # df = df[(df['xm'] =='京广高速') | (df['xm'] =='京广高速京西联')]
# df.reset_index(drop=True, inplace=True)
# df

In [ ]:
def delay_pct(x,var_in,var_out):
    totoal_freq = len(x[var_in])
    delay_freq = (x[var_in]>0).sum()
    x[var_out] = delay_freq/totoal_freq
    return x

def early_pct(x,var_in,var_out):
    totoal_freq = len(x[var_in])
    delay_freq = (x[var_in]<0).sum()
    x[var_out] = delay_freq/totoal_freq
    return x

df = df.groupby(['STN_NAME']).apply(delay_pct,'ARR_DELAY','stn_arr_delay_pct')
df = df.groupby(['ARR_NAME']).apply(delay_pct,'ARR_DELAY','train_arr_delay_pct')
df = df.groupby(['ARR_NAME','STN_NAME']).apply(delay_pct,'ARR_DELAY','stn_train_arr_delay_pct')

df = df.groupby(['STN_NAME']).apply(early_pct,'ARR_DELAY','stn_arr_early_pct')
df = df.groupby(['ARR_NAME']).apply(early_pct,'ARR_DELAY','train_arr_early_pct')
df = df.groupby(['ARR_NAME','STN_NAME']).apply(early_pct,'ARR_DELAY','stn_train_arr_early_pct')

df.head()

In [ ]:
def delay_ave(x,var_in,var_out):
    #totoal_freq = len(x[var_in])
    totoal_freq = len(x[var_in])
    totoal_time = ( x[var_in][x[var_in]>0] ).sum()
    x[var_out] = totoal_time/totoal_freq
    return x

def early_ave(x,var_in,var_out):
    totoal_freq = len(x[var_in])
    totoal_time = ( x[var_in][x[var_in]<0] ).sum()
    x[var_out] = abs(totoal_time)/totoal_freq
    return x

df = df.groupby(['STN_NAME']).apply(delay_ave,'ARR_DELAY','stn_arr_delay_ave')
df = df.groupby(['ARR_NAME']).apply(delay_ave,'ARR_DELAY','train_arr_delay_ave')
df = df.groupby(['ARR_NAME','STN_NAME']).apply(delay_ave,'ARR_DELAY','stn_train_arr_delay_ave')

df = df.groupby(['STN_NAME']).apply(early_ave,'ARR_DELAY','stn_arr_early_ave')
df = df.groupby(['ARR_NAME']).apply(early_ave,'ARR_DELAY','train_arr_early_ave')
df = df.groupby(['ARR_NAME','STN_NAME']).apply(early_ave,'ARR_DELAY','stn_train_arr_early_ave')

df[:100]

In [ ]:
# df['stn_tdtl_time'] = round(
#     (pd.to_datetime(df['DEPART0']) -
#      pd.to_datetime(df['ARRIVAL0'])).dt.seconds)  # 图定停站时间 / s
df['stn_tdtl_time']=np.where(df['ARRIVAL0']>df['DEPART0'],0,round(
    (pd.to_datetime(df['DEPART0']) -
     pd.to_datetime(df['ARRIVAL0'])).dt.seconds))
# df['STN_SJTL_TIME'] = round(
#     (pd.to_datetime(df['CHUFA_TIME']) -
#      pd.to_datetime(df['DAODA_TIME'])).dt.seconds)  # 实际停站时间 / s
df['STN_SJTL_TIME'] = np.where(df['DAODA_TIME']>df['CHUFA_TIME'],0,round(
    (pd.to_datetime(df['CHUFA_TIME']) -
     pd.to_datetime(df['DAODA_TIME'])).dt.seconds))  # 实际停站时间 / s
df['STN_XS_TIME'] = df['stn_tdtl_time'] - df['STN_SJTL_TIME']  # 站点吸收时间 / s

df.sort_values(['STATIS_ID', 'DAODA_TIME'],
               inplace=True,
               ascending=[True, True])
df.reset_index(drop=True, inplace=True)
df['FORMER_CHUFA_TIME']=datetime.datetime.now()
df['FORMER_DEPART1']=datetime.datetime.now()
df['FORMER_STN']='深圳'
df['FORMER_DPT_DELAY1']=0
# df['DAODA_TIME']=df['DAODA_TIME'].astype('datetime64')
for i in range(len(df)):
    if i==0:
        df['FORMER_CHUFA_TIME'].iat[i]=df['DAODA_TIME'].iat[i]
        df['FORMER_DEPART1'].iat[i]=df['ARRIVAL0'].iat[i]
        df['FORMER_STN'].iat[i]=df['STN_NAME'].iat[i]
        df['FORMER_DPT_DELAY1'].iat[i]=df['ARR_DELAY'].iat[i]
    else:
        if df['STATIS_ID'].iat[i]==df['STATIS_ID'].iat[i-1]:
            df['FORMER_CHUFA_TIME'].iat[i]=df['CHUFA_TIME'].iat[i-1]
            df['FORMER_DEPART1'].iat[i]=df['DEPART0'].iat[i-1]
            df['FORMER_STN'].iat[i]=df['STN_NAME'].iat[i-1]
            df['FORMER_DPT_DELAY1'].iat[i]=df['DPT_DELAY'].iat[i-1]
        else:
            df['FORMER_CHUFA_TIME'].iat[i]=df['DAODA_TIME'].iat[i]
            df['FORMER_DEPART1'].iat[i]=df['ARRIVAL0'].iat[i]
            df['FORMER_STN'].iat[i]=df['STN_NAME'].iat[i]
            df['FORMER_DPT_DELAY1'].iat[i]=df['ARR_DELAY'].iat[i]
#             df['FORMER_DPT_DELAY1'].iat[i]=df['ARR_DELAY'].iat[i]
# df['FORMER_CHUFA_TIME'] = df['CHUFA_TIME'].shift(1)  # 前站实际出发
# df['FORMER_DEPART1'] = df['DEPART0'].shift(1)  # 前站图定出发

# df['FORMER_STN'] = df['STN_NAME'].shift(1)  # 上一站
# df['FORMER_DPT_DELAY1'] = df['DPT_DELAY'].shift(1)  # *******

# df['ID_TMP'] = df['STATIS_ID'].shift(1)
# df.loc[
#     df['ID_TMP'] != df['STATIS_ID'],
#     ['FORMER_CHUFA_TIME', 'FORMER_DEPART1', 'FORMER_STN', 'FORMER_DPT_DELAY1'
#      ]] = np.nan


df['qujian_tdtl_time'] = round(
    (pd.to_datetime(df['ARRIVAL0']) -
     pd.to_datetime(df['FORMER_DEPART1'])).dt.seconds)  # 当前区间图定运行
df['QUJIAN_SJTL_TIME'] = round(
    (pd.to_datetime(df['DAODA_TIME']) -
     pd.to_datetime(df['FORMER_CHUFA_TIME'])).dt.seconds)  # 当前区间实际运行
df['QUJIAN_XS_TIME'] = df['qujian_tdtl_time'] - df[
    'QUJIAN_SJTL_TIME']  # 当前区间吸收

# df.drop(columns=['FORMER_CHUFA_TIME', 'FORMER_DEPART1', 'ID_TMP'],
#         inplace=True)

df.fillna(0, inplace=True)
df

In [ ]:
df=df[df['QUJIAN_SJTL_TIME']<10000]

In [ ]:
wt=df[df['qujian_tdtl_time']>80000]
wt=wt.drop_duplicates(['STATIS_ID'],keep='first')
df = df[~df['STATIS_ID'].isin(wt['STATIS_ID'])]

In [ ]:
# df

In [ ]:
df['yc']=df['ARR_DELAY']-df['FORMER_DPT_DELAY1']
yc=df[df['yc']<=-600]
yc=yc.drop_duplicates(['STATIS_ID'],keep='first')
df = df[~df['STATIS_ID'].isin(yc['STATIS_ID'])]
del df['yc']

In [ ]:
# df[df['STATIS_ID']==21882163.0]

In [ ]:
# df[df['QUJIAN_XS_TIME']>df['QUJIAN_SJTL_TIME']]
# def get_inital_dealy(df_init):
#     df['qc']=df_init['ARRIVAL0']-df_init['former_depart0']
# df1 = df.groupby(['STATIS_ID'],as_index=False).progress_apply(get_inital_dealy)

In [ ]:
# # df[(df['ARRIVAL0']-df['former_depart0'])]
# # df['ss']=df['ARRIVAL0']-df['former_depart0']
# df['ss'] = round(
#     (pd.to_datetime(df['ARRIVAL0']) -
#      pd.to_datetime(df['FORMER_DEPART1'])).dt.seconds)  # 当前区间图定运行

In [ ]:
def qujian_xs_ave(x,var_in,var_out):
    #totoal_freq = len(x[var_in])
    totoal_freq = len(x[var_in][x[var_in]>0])
    totoal_time = ( x[var_in][ (x[var_in]>0) & ( x['FORMER_DPT_DELAY1']>0) ] ).sum()
    x[var_out] = totoal_time/totoal_freq
    return x

def qujian_ks_ave(x,var_in,var_out):
    totoal_freq = len(x[var_in][x[var_in]<0])
    totoal_time = ( x[var_in][ (x[var_in]<0) & ( x['FORMER_DPT_DELAY1']>0) ] ).sum()
    x[var_out] = abs(totoal_time)/totoal_freq
    return x

def qujian_xs_pct(x,var_in,var_out):
    totoal_freq = len(x['FORMER_DPT_DELAY1']>0)
    delay_freq = ( (x[var_in]>0) & ( x['FORMER_DPT_DELAY1']>0) ).sum()
    x[var_out] = delay_freq/totoal_freq
    return x

def qujian_ks_pct(x,var_in,var_out):
    totoal_freq = len(x['FORMER_DPT_DELAY1']>0)
    delay_freq = ( (x[var_in]<0) & ( x['FORMER_DPT_DELAY1']>0) ).sum()
    x[var_out] = delay_freq/totoal_freq
    return x

# 某区间晚点吸收平均时间
df = df.groupby(['STN_NAME','FORMER_STN']).apply(qujian_xs_ave,'QUJIAN_XS_TIME','qujian_xs_ave')  
# 某车次在某区间晚点吸收平均时间
df = df.groupby(['ARR_NAME','STN_NAME','FORMER_STN']).apply(qujian_xs_ave,'QUJIAN_XS_TIME','train_qujian_xs_ave')  
#某区间晚点扩散平均时间
df = df.groupby(['STN_NAME','FORMER_STN']).apply(qujian_ks_ave,'QUJIAN_XS_TIME','qujian_ks_ave')
# 某车次在某区间晚点扩散平均时间
df = df.groupby(['ARR_NAME','STN_NAME','FORMER_STN']).apply(qujian_ks_ave,'QUJIAN_XS_TIME','train_qujian_ks_ave')  


# 某区间晚点吸收发生概率
df = df.groupby(['STN_NAME','FORMER_STN']).apply(qujian_xs_pct,'QUJIAN_XS_TIME','qujian_xs_pct')  
# 某车次在某区间晚点吸收发生概率
df = df.groupby(['ARR_NAME','STN_NAME','FORMER_STN']).apply(qujian_xs_pct,'QUJIAN_XS_TIME','train_qujian_xs_pct')  
#某区间晚点扩散发生概率
df = df.groupby(['STN_NAME','FORMER_STN']).apply(qujian_ks_pct,'QUJIAN_XS_TIME','qujian_ks_pct')
# 某车次在某区间晚点扩散发生概率
df = df.groupby(['ARR_NAME','STN_NAME','FORMER_STN']).apply(qujian_ks_pct,'QUJIAN_XS_TIME','train_qujian_ks_pct')  

In [ ]:
def stn_xs_ave(x,var_in,var_out):
    #totoal_freq = len(x[var_in])
    totoal_freq = len(x[var_in][x[var_in]>0])
    totoal_time = ( x[var_in][ (x[var_in]>0) & ( x['ARR_DELAY']>0) ] ).sum()
    x[var_out] = totoal_time/totoal_freq
    return x

def stn_ks_ave(x,var_in,var_out):
    totoal_freq = len(x[var_in][x[var_in]<0])
    totoal_time = ( x[var_in][ (x[var_in]<0) & ( x['ARR_DELAY']>0) ] ).sum()
    x[var_out] = abs(totoal_time)/totoal_freq
    return x
def stn_xs_pct(x,var_in,var_out):
    totoal_freq = len(x['ARR_DELAY']>0)
    delay_freq = ( (x[var_in]>0) & ( x['ARR_DELAY']>0) ).sum()
    x[var_out] = delay_freq/totoal_freq
    return x

def stn_ks_pct(x,var_in,var_out):
    totoal_freq = len(x['ARR_DELAY']>0)
    delay_freq = ( (x[var_in]<0) & ( x['ARR_DELAY']>0) ).sum()
    x[var_out] = delay_freq/totoal_freq
    return x

df = df.groupby(['ARR_NAME','STN_NAME']).apply(stn_xs_ave,'STN_XS_TIME','stn_train_xs_ave') 
df = df.groupby(['ARR_NAME','STN_NAME']).apply(stn_ks_ave,'STN_XS_TIME','stn_train_ks_ave') 

df = df.groupby(['ARR_NAME','STN_NAME']).apply(stn_xs_pct,'STN_XS_TIME','stn_train_xs_pct')  
df = df.groupby(['ARR_NAME','STN_NAME']).apply(stn_ks_pct,'STN_XS_TIME','stn_train_ks_pct')  

In [ ]:
# def qujian_delay_ave(x,var_in,var_out):
#     #totoal_freq = len(x[var_in])
#     totoal_freq = len(x[var_in][x[var_in]>0])
#     totoal_time = ( x[var_in][ (x[var_in]>0) & ( x['FORMER_DPT_DELAY1']>0) ] ).sum()
#     x[var_out] = totoal_time/totoal_freq
#     return x

# def qujian_early_ave(x,var_in,var_out):
#     totoal_freq = len(x[var_in][x[var_in]<0])
#     totoal_time = ( x[var_in][ (x[var_in]<0) & ( x['FORMER_DPT_DELAY1']>0) ] ).sum()
#     x[var_out] = abs(totoal_time)/totoal_freq
#     return x

# # 某区间晚点吸收平均时间
# df = df.groupby(['STN_NAME','FORMER_STN']).apply(qujian_delay_ave,'QUJIAN_XS_TIME','qujian_xs_ave')  
# # 某车次在某区间晚点吸收平均时间
# df = df.groupby(['ARR_NAME','STN_NAME','FORMER_STN']).apply(qujian_delay_ave,'QUJIAN_XS_TIME','train_qujian_xs_ave')  
# #某区间晚点扩散平均时间
# df = df.groupby(['STN_NAME','FORMER_STN']).apply(qujian_early_ave,'QUJIAN_XS_TIME','qujian_ks_ave')
# # 某车次在某区间晚点扩散平均时间
# df = df.groupby(['ARR_NAME','STN_NAME','FORMER_STN']).apply(qujian_early_ave,'QUJIAN_XS_TIME','train_qujian_ks_ave')  


# # 某区间晚点吸收发生概率
# df = df.groupby(['STN_NAME','FORMER_STN']).apply(delay_pct,'QUJIAN_XS_TIME','qujian_xs_pct')  
# # 某车次在某区间晚点吸收发生概率
# df = df.groupby(['ARR_NAME','STN_NAME','FORMER_STN']).apply(delay_pct,'QUJIAN_XS_TIME','train_qujian_xs_pct')  
# #某区间晚点扩散发生概率
# df = df.groupby(['STN_NAME','FORMER_STN']).apply(early_pct,'QUJIAN_XS_TIME','qujian_ks_pct')
# # 某车次在某区间晚点扩散发生概率
# df = df.groupby(['ARR_NAME','STN_NAME','FORMER_STN']).apply(early_pct,'QUJIAN_XS_TIME','train_qujian_ks_pct')  


In [ ]:
df['capacity'] = df.groupby('STN_NAME')['STN_NAME'].transform('count')
df

In [ ]:
# yc=df[df['STN_SJTL_TIME']>df['stn_tdtl_time']]
# yc.drop_duplicates('STATIS_ID',keep='first',inplace=True)
# yc
# df[df['ARR_DELAY'].shift(-1)-df['DPT_DELAY']]
# df['former_dep_delay'] = df['CHUFA_TIME'].shift(1)
# df['yc'] = round(
#     (pd.to_datetime(df['ARR_DELAY']) -
#      pd.to_datetime(df['FORMER_DPT_DELAY1'])).dt.seconds)  # 图定停站时间 / s

In [ ]:
df.sort_values(['STN_NAME','DAODA_TIME'],inplace=True,ascending=[True,True])
df.reset_index(drop=True, inplace=True) 
# df['front_train_arr_delay'] = df['ARR_DELAY'].shift(1)*60  # 前车当前站到达晚点
# df['front_train_dpt_delay'] = df['DPT_DELAY'].shift(1)*60 # 前车当前站出发晚点

print(df.shape)
# df

In [ ]:
months = pd.to_datetime(df['DEPART0']).dt.month

def assign_seasion(x):
    if x <= 3:
        return 'Spring'
    elif x <= 6:
        return 'Summer'
    elif x <= 9:
        return 'Autumn'
    else:
        return 'Winter'

season = months.progress_apply(assign_seasion)

df['Season'] = season
df

In [ ]:
"""
根据DEPART0获取日期，根据日期判断是工作日、休息日、节假日
dayofweek,小于4为Weekday（工作日），其他为Weekend（休息日）
节假日为法定节假日，优先级高于工作日和休息日
"""

dayofweek = pd.to_datetime(df['DEPART0']).dt.dayofweek

# 分配工作日、周末
def assign_day(x):
    if x <= 4:
        return 'Weekday'
    else:
        return 'Weekend'

whatday = dayofweek.progress_apply(assign_day)

df['WhatDay'] = whatday
df

In [ ]:
holidays = ['01-01', '02-04','02-05','02-06','02-07','02-08','02-09','02-10','04-05',
            '05-01','05-02','05-03','05-04','06-07','09-13', '10-01','10-02','10-03','10-04','10-05','10-06','10-07']

month_day = pd.to_datetime(df['DEPART0']).dt.strftime('%m-%d')
df['MonthDay'] = month_day

def assign_holiday(x):
    if x['MonthDay'] in holidays:
        return 'Holiday'
    else:
        return x['WhatDay']

df.loc[:, "WhatDay"] = df.progress_apply(assign_holiday, axis=1)

del df['MonthDay']
df

In [ ]:
hour = pd.to_datetime(df['DEPART0']).dt.hour

# 分配时间段
def assign_time(x):
    if x >= 0 and x < 6:
        return 'EarlyMorning'
    elif x >= 6 and x < 12:
        return 'Morning'
    elif x >= 12 and x < 18:
        return 'Afternoon'
    else:
        return 'Night'

what_time = hour.progress_apply(assign_time)

df['whattime'] = what_time
df

In [ ]:
# ARRIVAL0 , DEPART0 , AFTER_ARRIVAL0 , AFTER_DEPART0
"""
1.通通:B、C均为行调中间站。（图定到达时间 == 图定出发时间） #标记为1
2.发到:B、C均是客运中间站。（图定到达时间 != 图定出发时间） #标记为2
2.通到:B是行调中间站,C是客运中间站.(B 图定到达时间==图定出发时间,C图定到达时间!=图定出发时间)#标记为3
3.发通:B是客运中间站,C是行调中间站.(B 图定到达时间!=图定出发时间,C图定到达时间==图定出发时间)#标记为3
"""

# df['FORMER_ARRIVAL1'] = df['ARRIVAL0'].shift(1)
# df['FORMER_DEPART1'] = df['DEPART0'].shift(1)
df['FORMER_ARRIVAL1']=datetime.datetime.now()
df['FORMER_DEPART1']=datetime.datetime.now()
for i in range(len(df)):
    if i==0:
        df['FORMER_ARRIVAL1'].iat[i] = df['ARRIVAL0'].iat[i]
        df['FORMER_DEPART1'].iat[i]=df['ARRIVAL0'].iat[i]
    else:
        if df['STATIS_ID'].iat[i]==df['STATIS_ID'].iat[i-1]:
            df['FORMER_ARRIVAL1'].iat[i] = df['ARRIVAL0'].iat[i-1]
            df['FORMER_DEPART1']=df['DEPART0'].iat[i-1]
        else:
            df['FORMER_ARRIVAL1'].iat[i] = df['ARRIVAL0'].iat[i]
            df['FORMER_DEPART1'].iat[i]=df['ARRIVAL0'].iat[i]
conditions = [
    np.logical_and(df['ARRIVAL0'] == df['DEPART0'],
                   df['FORMER_ARRIVAL1'] == df['FORMER_DEPART1']),
    np.logical_and(df['ARRIVAL0'] == df['DEPART0'],
                   df['FORMER_ARRIVAL1'] != df['FORMER_DEPART1']),
    np.logical_and(df['ARRIVAL0'] != df['DEPART0'],
                   df['FORMER_ARRIVAL1'] == df['FORMER_DEPART1']),
    np.logical_and(df['ARRIVAL0'] != df['DEPART0'],
                   df['FORMER_ARRIVAL1'] != df['FORMER_DEPART1']),

]

choices = ['1', '2', '2', '3']
df.loc[:, 'qujian_yx_state'] = np.select(conditions, choices, default='unkown')

df.drop(columns=['FORMER_ARRIVAL1', 'FORMER_DEPART1'], inplace=True)
print(df.columns)
print(df['qujian_yx_state'].value_counts())
df

In [ ]:
# 区间吸收最大值
df['qujian_xs_max'] = df.groupby(['STN_NAME','FORMER_STN'])['QUJIAN_XS_TIME'].transform(lambda x: x.max())

# 车次区间吸收最大值
df['train_qujian_xs_max'] = df.groupby(['STN_NAME','FORMER_STN', 'ARR_NAME'])['QUJIAN_XS_TIME'].transform(lambda x: x.max())


df

In [ ]:
# 车站吸收最大值
df['stn_xs_max'] = df.groupby(['STN_NAME'])['STN_XS_TIME'].transform(lambda x: x.max())

# 车次车站吸收最大值
df['stn_train_xs_max'] = df.groupby(['STN_NAME','ARR_NAME'])['STN_XS_TIME'].transform(lambda x: x.max())

df

In [ ]:
# 获取每个行程（即每趟车）的下一站，用于判断前车和后车下一站是否相同
df["ARRIVAL0"] = pd.to_datetime(df["ARRIVAL0"], format= "%Y/%m/%d %H:%M:%S")
df.sort_values(['STATIS_ID', 'DAODA_TIME'], inplace=True, ascending=[True, True])
# df['NEXT_STN'] = df['STN_NAME'].shift(-1) # 当前列车下一站
df['NEXT_STN']='深圳'
for i in range(len(df)):
    if i==len(df)-1:
        df['NEXT_STN'].iat[i]=df['STN_NAME'].iat[i]
    else:
        if df['STATIS_ID'].iat[i]==df['STATIS_ID'].iat[i+1]:
            df['NEXT_STN'].iat[i]=df['STN_NAME'].iat[i+1]
        else:
            df['NEXT_STN'].iat[i]=df['STN_NAME'].iat[i]
    

df.sort_values(['STN_NAME', 'TRACK', 'NEXT_STN', 'DAODA_TIME'], inplace=True, ascending=[True, True, True, True])
df.reset_index(drop=True, inplace=True )

# df['FORMER_TRAIN'] = df['ARR_NAME'].shift(1)

# df['FORMER_TRAIN_NEXT_STN'] = df['NEXT_STN'].shift(1)
# df['FORMER_TRAIN_ARRIVAL1'] = df['ARRIVAL0'].shift(1)
# df['FORMER_TRAIN'] ='G'
# df['FORMER_TRAIN_NEXT_STN']='深圳'
# df['FORMER_TRAIN_ARRIVAL1']=datetime.datetime.now()
# for i in range(len(df)):
#     if i==0:
#         df['FORMER_TRAIN'].iat[i]=df['ARR_NAME'].iat[i]
#         df['FORMER_TRAIN_NEXT_STN'].iat[i]=df['NEXT_STN'].iat[i]
#         df['FORMER_TRAIN_ARRIVAL1'].iat[i]=df['ARRIVAL0'].iat[i]
#     else:
#         if df['STATIS_ID'].iat[i]==df['STATIS_ID'].iat[i-1]:
#             df['FORMER_TRAIN'].iat[i]=df['ARR_NAME'].iat[i-1]
#             df['FORMER_TRAIN_NEXT_STN'].iat[i]=df['NEXT_STN'].iat[i-1]
#             df['FORMER_TRAIN_ARRIVAL1'].iat[i]=df['ARRIVAL0'].iat[i-1]
#         else:
#             df['FORMER_TRAIN'].iat[i]=df['ARR_NAME'].iat[i]
#             df['FORMER_TRAIN_NEXT_STN'].iat[i]=df['NEXT_STN'].iat[i]
#             df['FORMER_TRAIN_ARRIVAL1'].iat[i]=df['ARRIVAL0'].iat[i]
# current_time = df['ARRIVAL0']
# former_time = df['FORMER_TRAIN_ARRIVAL1']

# diff = (current_time - former_time)
# diff = diff.fillna(0)
# diff = diff.dt.total_seconds().astype(int)
# df['arr_diff'] = diff

# # 站点/轨道/下一站变化时不是前后车关系
# # mask = np.logical_and.reduce((df['STN_NAME'].shift(1) == df['STN_NAME'], 
# #                               df['TRACK'].shift(1) == df['TRACK'],
# #                               df['FORMER_TRAIN_NEXT_STN'].shift(-1) == df['FORMER_TRAIN_NEXT_STN']))

# # df['arr_diff'] = np.where(mask, df['arr_diff'], 0)
# # df['arr_diff'] = np.where(df['arr_diff'] < 0 , 0, df['arr_diff'])

# df.drop(df[['NEXT_STN','FORMER_TRAIN','FORMER_TRAIN_NEXT_STN','FORMER_TRAIN_ARRIVAL1']], axis=1, inplace=True)

# print(df.info())
# df

In [ ]:
# df2 = pd.read_csv(os.path.join(gbk_data_path, 'ZM_table_full.csv'))
df2 = pd.read_csv('/home/AI/other/ZM_table_full.csv')
print(df2.shape)
lc_map = {(k1,k2): v for k1, k2, v in df2[['Q_CZM','H_CZM','ZXLC']].values}


def assign_lc(x):
    k1 = x['FORMER_STN']
    k2 = x['STN_NAME']
    try:
        return lc_map[(k1, k2)]
    except:
        try: 
            return lc_map[(k2, k1)]
        except:
            return 0 

df.loc[:, "DISTANCE"] = df.progress_apply(assign_lc, axis=1)

df['PLAN_SPEED'] = df['DISTANCE'] / df['qujian_tdtl_time'] * 3600

print(np.sum((df["DISTANCE"]==0) & (df['FORMER_STN']!=0))/len(df))
df

In [ ]:
# 异常速度值截断为350km/h
df['PLAN_SPEED'] = np.where(df['PLAN_SPEED']>350, 350, df['PLAN_SPEED'])

df

In [ ]:
# df1=df[df['XM']=='京沪高铁']
# df

In [ ]:
df.sort_values(['STATIS_ID', 'DAODA_TIME'], inplace=True, ascending=[True, True])
df

In [ ]:
df.reset_index(inplace=True,drop=True)
df

In [ ]:
def get_inital_dealy(df_init):
    
    df_init.reset_index(inplace=True,drop=True)
#     print(df_init['STATIS_ID'][0])
#     print(df_init.shape)
    df_init['seq_id_arr'] = '0'
    df_init['seq_id_dpt'] = '0'
    arr_delay_list = list(np.where((df_init['ARR_DELAY_TYPE'] == '到达初始晚点'))[0])
    arr_delay_list.append(df_init.index[-1])
    dpt_delay_list = list(np.where((df_init['DPT_DELAY_TYPE'] == '出发初始晚点'))[0])
    dpt_delay_list.append(df_init.index[-1])
#     print('arr_delay_list',arr_delay_list)
#     print('arr_delay_list',dpt_delay_list)
    
    if np.sum(arr_delay_list) >= 1:
#         判断最后一战是否为初始晚点，避免数据重复
#         print('flag', (df_init.iloc[-1]['ARR_DELAY_TYPE'] == '到达初始晚点'))
#         if not (df_init.iloc[-1]['ARR_DELAY_TYPE'] == '到达初始晚点'):
#             arr_delay_list.append(df_init.index[-1])
            
#         print('arr_delay_list',arr_delay_list)
        for i in range(len(arr_delay_list) - 1):
            arr_start_index = arr_delay_list[i]
            arr_end_index = arr_delay_list[i + 1]
            
            empty_seq_arr = df_init.loc[arr_start_index+1:arr_end_index, ['ARR_DELAY_TYPE', 'DPT_DELAY_TYPE']]
            arr_end_index_df = empty_seq_arr[((empty_seq_arr['ARR_DELAY_TYPE'] == '到达正点') | (empty_seq_arr['ARR_DELAY_TYPE'] == '早到')) 
                                             | ((empty_seq_arr['DPT_DELAY_TYPE'] == '出发正点') |  (empty_seq_arr['DPT_DELAY_TYPE'] == '早发'))]
#             print('arr_start_index',arr_start_index)
#             print('arr_end_index',arr_end_index)
#             print('empty_seq_arr',empty_seq_arr)
            if not arr_end_index_df.empty:
                arr_end_index = arr_end_index_df.index[0]
               
#             print('arr_end_index',arr_end_index)
    
            df_init['seq_id_arr'][arr_start_index:arr_end_index+1] = str(uuid.uuid1())
            
    if np.sum(dpt_delay_list) >= 1:
        # 判断最后一战是否为初始晚点，避免数据重复
#         if not (df_init.iloc[-1]['DPT_DELAY_TYPE'] == '出发初始晚点'):
#             dpt_delay_list.append(df_init.index[-1])
#         print('dpt_delay_list',dpt_delay_list)
        for i in range(len(dpt_delay_list) - 1):
            dpt_start_index = dpt_delay_list[i]
            dpt_end_index = dpt_delay_list[i + 1]
            
            empty_seq_dpt = df_init.loc[dpt_start_index+1:dpt_end_index, ['ARR_DELAY_TYPE', 'DPT_DELAY_TYPE']]
            dpt_end_index_df = empty_seq_dpt[((empty_seq_dpt['ARR_DELAY_TYPE'] == '到达正点') | (empty_seq_dpt['ARR_DELAY_TYPE'] == '早到')) 
                                             | ((empty_seq_dpt['DPT_DELAY_TYPE'] == '出发正点') |  (empty_seq_dpt['DPT_DELAY_TYPE'] == '早发'))]
#             print('dpt_start_index',dpt_start_index)
#             print('dpt_end_index',dpt_end_index)
#             print('empty_seq_dpt',empty_seq_dpt)
            if not dpt_end_index_df.empty:
                dpt_end_index = dpt_end_index_df.index[0]
#             print('dpt_end_index',dpt_end_index)
            
            df_init['seq_id_dpt'][dpt_start_index:dpt_end_index+1] = str(uuid.uuid1())
            
    return df_init


In [ ]:
# df.groupby(['STATIS_ID'],as_index=False).progress_apply(get_inital_dealy)
# get_inital_dealy(df[df['STATIS_ID'] ==  21770071])

df = df.groupby(['STATIS_ID'],as_index=False).progress_apply(get_inital_dealy)
df.reset_index(inplace=True, drop=True)

df['seq_id_arr'] = df['seq_id_arr'].astype("category")
df['seq_id_dpt'] = df['seq_id_dpt'].astype("category")

df['seq_id_arr'] = df['seq_id_arr'].cat.rename_categories(range(len(df['seq_id_arr'].unique())))
df['seq_id_dpt'] = df['seq_id_dpt'].cat.rename_categories(range(len(df['seq_id_dpt'].unique())))
df

In [ ]:
arr=df[(df['seq_id_arr']!=0) & (df['seq_id_arr']!='0')]
arr

In [ ]:
dpt=df[(df['seq_id_dpt']!=0) & (df['seq_id_dpt']!='0')]
dpt

In [ ]:
dawandian=arr[(arr['ARR_DELAY_TYPE']=='到达初始晚点') & (arr['ARR_DELAY']>=300) ]['seq_id_arr']
print(len(dawandian))
dawandian=arr[arr['seq_id_arr'].isin(dawandian)]

In [ ]:
dawandian

In [ ]:
xiao=arr[(arr['ARR_DELAY_TYPE']=='到达初始晚点') & (arr['ARR_DELAY']<300) ]['seq_id_arr']

In [ ]:
shuju=df[df['seq_id_arr'].isin(xiao)]

In [ ]:
shuju=shuju[(shuju['ARR_DELAY_TYPE']=='到达连带晚点')|(shuju['DPT_DELAY_TYPE']=='出发连带晚点')]
a=shuju[(shuju['ARR_DELAY']>=300)|(shuju['DPT_DELAY']>=300)]['seq_id_arr'].unique()
len(a)

In [ ]:
xiaowandian=arr[arr['seq_id_arr'].isin(a)]
xiaowandian

In [ ]:
wandian=pd.concat([dawandian,xiaowandian])
wandian

In [ ]:
da=wandian[(wandian['ARR_DELAY_TYPE']=='到达初始晚点') &(wandian['ARR_DELAY']>1800)]['seq_id_arr']
wandian=wandian[~wandian['seq_id_arr'].isin(da)]

In [ ]:
da=dawandian[(dawandian['ARR_DELAY_TYPE']=='到达初始晚点') &(dawandian['ARR_DELAY']>1800)]['seq_id_arr']
wandian=dawandian[~dawandian['seq_id_arr'].isin(da)]

In [ ]:
# da=wandian[(wandian['ARR_DELAY']>=1620)| (wandian['DPT_DELAY']>=1620)]['seq_id_arr']
# wandian=dawandian[~dawandian['seq_id_arr'].isin(da)]

In [ ]:
wandian.sort_values(['STATIS_ID', 'DAODA_TIME'], inplace=True, ascending=[True, True])
wandian

In [ ]:
wandian.reset_index(inplace=True,drop=True)
wandian

In [ ]:
wandian[wandian['STN_FLAG']=='1']

In [ ]:
data=arr[(arr['ARR_DELAY_TYPE']=='到达初始晚点')&( (arr['ARR_DELAY']>=60) & (arr['ARR_DELAY']<=240) )]['seq_id_arr']
data=arr[arr['seq_id_arr'].isin(data)]
data.sort_values(['STATIS_ID', 'DAODA_TIME'], inplace=True, ascending=[True, True])
data.reset_index(inplace=True,drop=True)
data

In [ ]:
qc=data.drop_duplicates(['seq_id_arr'],keep='first')
qc=qc['seq_id_arr']
lb=[]
for i in qc:
    sj=data[data['seq_id_arr']==i]
    lb.append(len(sj))
print(len(lb))

In [ ]:
qq=data.drop_duplicates(['STATIS_ID'],keep='first')
qq

In [ ]:
data1=arr[(arr['ARR_DELAY_TYPE']=='到达初始晚点')&( (arr['ARR_DELAY']>=300) & (arr['ARR_DELAY']<=600) )]['seq_id_arr']
data1=arr[arr['seq_id_arr'].isin(data1)]
data1.sort_values(['STATIS_ID', 'DAODA_TIME'], inplace=True, ascending=[True, True])
data1.reset_index(inplace=True,drop=True)
data1
qc1=data1.drop_duplicates(['seq_id_arr'],keep='first')
qc1=qc1['seq_id_arr']
lb1=[]
for i in qc1:
    sj=data1[data1['seq_id_arr']==i]
    lb1.append(len(sj)) 
print(len(lb1))

In [ ]:
data1=arr[(arr['ARR_DELAY_TYPE']=='到达初始晚点')&( (arr['ARR_DELAY']>1800))]['seq_id_arr']
data1=arr[arr['seq_id_arr'].isin(data1)]
data1.sort_values(['STATIS_ID', 'DAODA_TIME'], inplace=True, ascending=[True, True])
data1.reset_index(inplace=True,drop=True)
data1
qc1=data1.drop_duplicates(['seq_id_arr'],keep='first')
qc1=qc1['seq_id_arr']
lb1=[]
for i in qc1:
    sj=data1[data1['seq_id_arr']==i]
    lb1.append(len(sj)) 
print(len(lb1))

In [ ]:
data2=arr[(arr['ARR_DELAY_TYPE']=='到达初始晚点')&( (arr['ARR_DELAY']>=660) & (arr['ARR_DELAY']<=1800) )]['seq_id_arr']
data2=arr[arr['seq_id_arr'].isin(data2)]
data2.sort_values(['STATIS_ID', 'DAODA_TIME'], inplace=True, ascending=[True, True])
data2.reset_index(inplace=True,drop=True)
qc2=data2.drop_duplicates(['seq_id_arr'],keep='first')
qc2=qc2['seq_id_arr']
lb2=[]
for i in qc2:
    sj=data2[data2['seq_id_arr']==i]
    lb2.append(len(sj)) 
print(len(lb2))

In [ ]:
# plt.sca(ax1)
plt.figure(dpi=150)
# plt.xlabel('1-5')
s1=pd.Series(np.array(lb))
s2=pd.Series(np.array(lb1))
s3=pd.Series(np.array(lb2))
yes=pd.DataFrame({'1-4':s1,'5-10':s2,'11-30':s3})
# plt.subplot(1,1,2)
yes.boxplot()
# # plt.subplot(2,2,2)
# # plt.boxplot(lb1)
# # plt.subplot(2,2,3)
# # plt.boxplot(lb2)
plt.savefig('箱型图.jpg',bbox_inches='tight')
plt.show()


In [ ]:
yes.to_csv('画图.csv',index=None)

In [ ]:
plt.figure(dpi=150)
plt.xlabel('6-10')
# plt.subplot(2,2,1)
plt.boxplot(lb1)
# plt.subplot(2,2,2)
# plt.boxplot(lb1)
# plt.subplot(2,2,3)
# plt.boxplot(lb2)
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.xlabel('11-30')
# plt.subplot(2,2,1)
plt.boxplot(lb2)
# plt.subplot(2,2,2)
# plt.boxplot(lb1)
# plt.subplot(2,2,3)
# plt.boxplot(lb2)
plt.show()

In [ ]:
wandian[(wandian['ARRIVAL0']!=wandian['DEPART0'])& (wandian['ARR_DELAY']!=wandian['DPT_DELAY'])]

In [ ]:
wandian[wandian['STATIS_ID']==21756007.0]

In [ ]:
columns=['STATIS_ID','FORMER_DPT_DELAY1','ARR_DELAY','qujian_tdtl_time','QUJIAN_SJTL_TIME','DAODA_TIME','FORMER_CHUFA_TIME']
j=wandian[columns]
j['xishou']=(j['FORMER_DPT_DELAY1']-j['ARR_DELAY'])
j[np.abs(j['xishou'])>300]
# [np.abs(j['FORMER_DPT_DELAY1']-j['ARR_DELAY'])>300]
# j[j['QUJIAN_SJTL_TIME']>1800]

In [ ]:
df[df['STATIS_ID']==28916877.0]

In [ ]:
January=wandian[(wandian['ARRIVAL0'].dt.month)==1]
January.drop_duplicates('STATIS_ID',keep='first',inplace=True)
January=len(January['STATIS_ID'])
February=wandian[(wandian['ARRIVAL0'].dt.month)==2]
February.drop_duplicates('STATIS_ID',keep='first',inplace=True)
February=len(February['STATIS_ID'])
March=wandian[(wandian['ARRIVAL0'].dt.month)==3]
March.drop_duplicates('STATIS_ID',keep='first',inplace=True)
March=len(March['STATIS_ID'])
April=wandian[(wandian['ARRIVAL0'].dt.month)==4]
April.drop_duplicates('STATIS_ID',keep='first',inplace=True)
April=len(April['STATIS_ID'])
May=wandian[(wandian['ARRIVAL0'].dt.month)==5]
May.drop_duplicates('STATIS_ID',keep='first',inplace=True)
May=len(May['STATIS_ID'])
June=wandian[(wandian['ARRIVAL0'].dt.month)==6]
June.drop_duplicates('STATIS_ID',keep='first',inplace=True)
June=len(June['STATIS_ID'])
July=wandian[(wandian['ARRIVAL0'].dt.month)==7]
July.drop_duplicates('STATIS_ID',keep='first',inplace=True)
July=len(July['STATIS_ID'])
August=wandian[(wandian['ARRIVAL0'].dt.month)==8]
August.drop_duplicates('STATIS_ID',keep='first',inplace=True)
August=len(August['STATIS_ID'])
September=wandian[(wandian['ARRIVAL0'].dt.month)==9]
September.drop_duplicates('STATIS_ID',keep='first',inplace=True)
September=len(September['STATIS_ID'])
October=wandian[(wandian['ARRIVAL0'].dt.month)==10]
October.drop_duplicates('STATIS_ID',keep='first',inplace=True)
October=len(October['STATIS_ID'])
November=wandian[(wandian['ARRIVAL0'].dt.month)==11]
November.drop_duplicates('STATIS_ID',keep='first',inplace=True)
November=len(November['STATIS_ID'])
December=wandian[(wandian['ARRIVAL0'].dt.month)==12]
December.drop_duplicates('STATIS_ID',keep='first',inplace=True)
December=len(December['STATIS_ID'])

In [ ]:
month=[January,February,March,April,May,June,July,August,September,October,November,December]
month

In [ ]:
name=['1','2','3','4','5','6','7','8','9','10','11','12']
plt.bar(name,month,color='rgby')
for a,b in zip(name,month):
    plt.text(a,b,'%.0f' % b,ha='center',va='bottom',fontsize=11)
    
plt.savefig("沪蓉月")
plt.show()


In [ ]:
one=wandian[((wandian['ARRIVAL0'].dt.hour)>=6) & ((wandian['ARRIVAL0'].dt.hour)<=11)]
one.drop_duplicates('STATIS_ID',keep='first',inplace=True)
one=len(one['STATIS_ID'])

two=wandian[((wandian['ARRIVAL0'].dt.hour)>=12) & ((wandian['ARRIVAL0'].dt.hour)<=17)]
two.drop_duplicates('STATIS_ID',keep='first',inplace=True)
two=len(two['STATIS_ID'])

three=wandian[((wandian['ARRIVAL0'].dt.hour)>=18) & ((wandian['ARRIVAL0'].dt.hour)<=24)]
three.drop_duplicates('STATIS_ID',keep='first',inplace=True)
three=len(three['STATIS_ID'])


In [ ]:
hour=[one,two,three]

name=['6-11','12-17','18-24']
plt.bar(name,hour,color='rgby',width=0.4)
for a,b in zip(name,hour):
    plt.text(a,b,'%.0f' % b,ha='center',va='bottom',fontsize=11)
    
plt.savefig("沪蓉小时")
plt.show()

In [ ]:
df[df['seq_id_arr'].isin(a)]

In [ ]:
print(len(arr['STATIS_ID'].unique()))

In [ ]:
wandian.drop_duplicates('seq_id_arr',keep='first',inplace=True)
wandian

In [ ]:
dpt=df[df['seq_id_dpt']!=0]
dpt

In [ ]:
wandian[((wandian['ARR_DELAY']>=300)&(wandian['ARR_DELAY']<=1800)) & (wandian['ARR_DELAY_TYPE']=='到达初始晚点')]

In [ ]:
dawandian

In [ ]:
len(dpt[(dpt['DPT_DELAY_TYPE']=='出发初始晚点') & (dpt['DPT_DELAY']>=300) ]['seq_id_dpt'])

In [ ]:
xiao1=dpt[(dpt['DPT_DELAY_TYPE']=='出发初始晚点') & (dpt['DPT_DELAY']<300) ]['seq_id_dpt']
len(xiao1)

In [ ]:
shuju1=df[df['seq_id_dpt'].isin(xiao1)]
shuju1=shuju1[(shuju1['ARR_DELAY_TYPE']=='到达连带晚点')|(shuju1['DPT_DELAY_TYPE']=='出发连带晚点')]
b=shuju1[(shuju1['ARR_DELAY']>=300)|(shuju1['DPT_DELAY']>=300)]['seq_id_dpt'].unique()
len(b)

In [ ]:
df[~df['seq_id_dpt'].isin(b)]

In [ ]:
print(len(dpt['STATIS_ID'].unique()))

In [ ]:
origin2[origin2['STATIS_ID']==21759209.0]

In [ ]:
dpt[dpt['STATIS_ID']==21759209.0]

In [ ]:
dpt[dpt['STATIS_ID']==21773781.0]

In [ ]:
df=df[df['seq_id_arr']!=0]
df

In [ ]:
df.to_csv(gbk_data_path+'/2020_ARR_xulie.csv', index=None)

In [ ]:
wandian.to_csv('2020_ARR_hkdwandian.csv', index=None)

In [ ]:
df['ARRIVAL1'].max()

In [ ]:
df['ARRIVAL1'].min()

In [ ]:
arr.to_csv(gbk_data_path+'/2020_jh_jz.csv', index=None)

In [ ]:
df[df['STATIS_ID']==26439803.0]

In [ ]:

ID['LAST_DPT_DELAY'] = ID['DPT_DELAY1'].shift(1)
ID['LAST_DPT_DELAY'] = np.where(ID['STATIS_ID'].shift(1) == ID['STATIS_ID'], ID['LAST_DPT_DELAY'], 0)  # 始发站上一站出发晚点时间置0
ID['ARR_DELAY1_type']='0'

In [ ]:
def get_inital_dealy(df_init):
    arr = list(np.where((df_init['ARR_DELAY1']>4))[0])
    dpt = list(np.where((df_init['DPT_DELAY1']>4))[0])
    for i in range(len(df_init)):
        if df_init['ARR_DELAY1']>4:
            df_init['ARR_DELAY1_type']='到达初始晚点'
#         df_init['ARR_DELAY1_type'].iloc[arr]='到达初始晚点'
#         if  df_init['ARR_DELAY1']<=4:
#              df_init['ARR_DELAY1_type']='到达正点'
        
    print(df_init['ARR_DELAY1_type'])    
origin1 = ID.groupby(['STATIS_ID'], as_index=False).progress_apply(get_inital_dealy)
origin1.reset_index(drop=True,inplace=True)


In [ ]:
# 构建"上一站出发晚点时间"字段
df['LAST_DPT_DELAY'] = df['DPT_DELAY1'].shift(1)
df['LAST_DPT_DELAY'] = np.where(df['STATIS_ID'].shift(1) == df['STATIS_ID'], df['LAST_DPT_DELAY'], 0)  # 始发站上一站出发晚点时间置0

# 构建"到达晚点类型"字段
conditions = [df['ARR_DELAY1'] <= 4,df['ARR_DELAY1'] > 4]
choices = ['到达正点','到达初始晚点']  # 先将晚点全部设为初始晚点，后续再部分更新为连带晚点
df['ARR_DELAY_TYPE'] = np.select(conditions, choices, default='unkown')

# 构建"出发晚点类型"字段
conditions = [df['DPT_DELAY1'] <= 4, df['DPT_DELAY1'] > 4]
choices = ['出发正点','出发初始晚点']
df['DPT_DELAY_TYPE'] = np.select(conditions, choices, default='unkown')